In [1]:
# Install the required Python libraries:
# - yfinance: For financial data analysis and stock information retrieval.
# - langchain_pinecone: For integrating LangChain with Pinecone, enabling vector database functionality.
# - python-dotenv: To manage environment variables stored in a .env file.
# - langchain-community: Community-contributed tools for LangChain.
# - sentence_transformers: For embedding text into vector space using pre-trained models.

! pip install yfinance langchain_pinecone python-dotenv langchain-community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.9
    Uninstalling aiohttp-3.11.9:
      Successfully uninstalled aiohttp-3.11.9


In [2]:
# Import PineconeVectorStore from langchain_pinecone for managing vector-based data storage and retrieval.
from langchain_pinecone import PineconeVectorStore
# Import OpenAI for interacting with OpenAI's API (e.g., GPT models).
from openai import OpenAI
# Import dotenv to load environment variables from a .env file securely.
import dotenv
# Import json for handling JSON data (e.g., parsing or generating JSON).
import json
# Import yfinance (aliased as yf) for accessing financial data such as stock prices and historical trends.
import yfinance as yf
# Import concurrent.futures to enable parallel execution of tasks using threads or processes.
import concurrent.futures
# Import HuggingFaceEmbeddings from langchain_community for embedding text using Hugging Face models.
from langchain_community.embeddings import HuggingFaceEmbeddings
# Import userdata from google.colab for handling user-specific data within Google Colab.
from google.colab import userdata
# Import Document from langchain.schema to represent structured documents in LangChain.
from langchain.schema import Document
# Import cosine_similarity from sklearn.metrics.pairwise to calculate the similarity between vectors.
from sklearn.metrics.pairwise import cosine_similarity
# Import SentenceTransformer from sentence_transformers for embedding text into vector space using pre-trained models.
from sentence_transformers import SentenceTransformer
# Import Pinecone from pinecone for connecting to the Pinecone vector database service.
from pinecone import Pinecone
# Import numpy (aliased as np) for numerical operations on arrays and matrices.
import numpy as np
# Import requests for making HTTP requests to APIs or web resources.
import requests
# Import os for interacting with the operating system (e.g., file paths or environment variables).
import os

In [3]:
def get_stock_info(symbol: str) -> dict:
    """
    Fetches detailed stock information from Yahoo Finance.

    Args:
        symbol (str): Stock ticker symbol.

    Returns:
        dict: Stock details like ticker, name, summary, location, industry, and sector.
    """
    data = yf.Ticker(symbol)
    stock_info = data.info

    return {
        "Ticker": stock_info.get('symbol', 'N/A'),
        'Name': stock_info.get('longName', 'N/A'),
        'Business Summary': stock_info.get('longBusinessSummary'),
        'City': stock_info.get('city', 'N/A'),
        'State': stock_info.get('state', 'N/A'),
        'Country': stock_info.get('country', 'N/A'),
        'Industry': stock_info.get('industry', 'N/A'),
        'Sector': stock_info.get('sector', 'N/A')
    }

In [4]:
data = yf.Ticker("NVDA")
stock_info = data.info
print(stock_info)

{'address1': '2788 San Tomas Expressway', 'city': 'Santa Clara', 'state': 'CA', 'zip': '95051', 'country': 'United States', 'phone': '408 486 2000', 'website': 'https://www.nvidia.com', 'industry': 'Semiconductors', 'industryKey': 'semiconductors', 'industryDisp': 'Semiconductors', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': "NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for gaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU or vGPU software for cloud-based visual and virtual computing; automotive platforms for infotainment systems; and Omniverse software for building and operating metaverse and 3D internet applications. The Compute & Networking segment co

In [5]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    """
    Generates embeddings for text using a Hugging Face model.

    Args:
        text (str): Input text.
        model_name (str): Hugging Face model name (default: "sentence-transformers/all-mpnet-base-v2").

    Returns:
        np.ndarray: Text embeddings as a NumPy array.
    """
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    """
    Computes cosine similarity between two sentences.

    Args:
        sentence1 (str): First sentence.
        sentence2 (str): Second sentence.

    Returns:
        float: Cosine similarity score (-1 to 1).

    Notes:
        Prints the similarity score.
    """
    # Get embeddings for sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape and calculate similarity
    similarity = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))
    similarity_score = similarity[0][0]
    print(f"Cosine similarity: {similarity_score:.4f}")
    return similarity_score


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like running to the office"

similarity = cosine_similarity_between_sentences(sentence1, sentence2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Cosine similarity: 0.6133


In [6]:
aapl_info = get_stock_info('AAPL')
print(aapl_info)

{'Ticker': 'AAPL', 'Name': 'Apple Inc.', 'Business Summary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts, as well as advertising services include third-party licensing arrangements and its own advertising platforms. In addition, the company offers various subscription-based services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experien

In [7]:
aapl_description = aapl_info['Business Summary']

company_description = "I want to find companies that make smartphones and are headquarted in California"

similarity = cosine_similarity_between_sentences(aapl_description, company_description)

Cosine similarity: 0.3635


In [8]:
def get_company_tickers():
    """
    Fetches company ticker symbols from an SEC JSON file on GitHub.

    Returns:
        dict: Dictionary of company tickers and related info.

    Notes:
        Data source:
        https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json
    """
    # URL of the JSON file
    url = "https://raw.githubusercontent.com/team-headstart/Financial-Analysis-and-Automation-with-LLMs/main/company_tickers.json"

    # Fetch the file
    response = requests.get(url)

    # Check for success
    if response.status_code == 200:
        # Parse JSON content
        company_tickers = json.loads(response.content.decode('utf-8'))

        # Save to local file
        with open("company_tickers.json", "w", encoding="utf-8") as file:
            json.dump(company_tickers, file, indent=4)

        print("File downloaded and saved as 'company_tickers.json'")
        return company_tickers
    else:
        print(f"Download failed. Status code: {response.status_code}")
        return None

company_tickers = get_company_tickers()

File downloaded and saved as 'company_tickers.json'


In [9]:
company_tickers

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '3': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '4': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '5': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '6': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '7': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '8': {'cik_str': 1046179,
  'ticker': 'TSM',
  'title': 'TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD'},
 '9': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '13': {'cik_str'

In [10]:
len(company_tickers)

9998

In [11]:
# Retrieve Pinecone API key from user data and set it as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Define the Pinecone index name and namespace
index_name = "stocks"
namespace = "stock-descriptions"

# Initialize Hugging Face embeddings
hf_embeddings = HuggingFaceEmbeddings()

# Create a Pinecone vector store with the specified index and embeddings
vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)


<ipython-input-11-e53b54897cb4>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings()
<ipython-input-11-e53b54897cb4>:10: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hf_embeddings = HuggingFaceEmbeddings()


In [12]:
# Initialize lists to track processing results
successful_tickers = []
unsuccessful_tickers = []

# Attempt to load previously successful tickers from file
try:
    with open('successful_tickers.txt', 'r') as f:
        # Read and clean lines from the file
        successful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(successful_tickers)} successful tickers")
except FileNotFoundError:
    # Handle case where the file doesn't exist
    print("No existing successful tickers file found")

# Attempt to load previously unsuccessful tickers from file
try:
    with open('unsuccessful_tickers.txt', 'r') as f:
        # Read and clean lines from the file
        unsuccessful_tickers = [line.strip() for line in f if line.strip()]
    print(f"Loaded {len(unsuccessful_tickers)} unsuccessful tickers")
except FileNotFoundError:
    # Handle case where the file doesn't exist
    print("No existing unsuccessful tickers file found")


No existing successful tickers file found
No existing unsuccessful tickers file found


In [13]:
def process_stock(stock_ticker: str) -> str:
    # Skip if the stock has already been processed
    if stock_ticker in successful_tickers:
        return f"Already processed {stock_ticker}"

    try:
        # Fetch stock data and description
        stock_data = get_stock_info(stock_ticker)
        stock_description = stock_data['Business Summary']

        # Store the stock description in Pinecone
        vectorstore_from_texts = PineconeVectorStore.from_documents(
            documents=[Document(page_content=stock_description, metadata=stock_data)],
            embedding=hf_embeddings,
            index_name=index_name,
            namespace=namespace
        )

        # Log successful processing
        with open('successful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        successful_tickers.append(stock_ticker)

        return f"Processed {stock_ticker} successfully"

    except Exception as e:
        # Log unsuccessful processing
        with open('unsuccessful_tickers.txt', 'a') as f:
            f.write(f"{stock_ticker}\n")
        unsuccessful_tickers.append(stock_ticker)

        return f"ERROR processing {stock_ticker}: {e}"

def parallel_process_stocks(tickers: list, max_workers: int = 10) -> None:
    # Use a thread pool to process stocks in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Map each stock to a future task
        future_to_ticker = {
            executor.submit(process_stock, ticker): ticker
            for ticker in tickers
        }

        # Process completed tasks
        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker = future_to_ticker[future]
            try:
                result = future.result()
                print(result)

                # Stop the program if an error occurs
                if result.startswith("ERROR"):
                    print(f"Stopping program due to error in {ticker}")
                    executor.shutdown(wait=False)
                    raise SystemExit(1)

            except Exception as exc:
                # Handle unexpected exceptions
                print(f'{ticker} generated an exception: {exc}')
                print("Stopping program due to exception")
                executor.shutdown(wait=False)
                raise SystemExit(1)

# Extract stock tickers from the company_tickers dictionary
tickers_to_process = [company_tickers[num]['ticker'] for num in company_tickers.keys()]

# Process the tickers in parallel
parallel_process_stocks(tickers_to_process, max_workers=10)

Processed NVDA successfully
Processed AAPL successfully
Processed BRK-B successfully
Processed AMZN successfully
Processed MSFT successfully
Processed TSLA successfully
Processed META successfully
Processed GOOGL successfully
Processed AVGO successfully
Processed TSM successfully
Processed LLY successfully
Processed JPM successfully
Processed WMT successfully
Processed UNH successfully
Processed V successfully
Processed SPY successfully
Processed HD successfully
Processed ORCL successfully
Processed XOM successfully
Processed NVO successfully
Processed MA successfully
Processed COST successfully
Processed JNJ successfully
Processed NFLX successfully
Processed BAC successfully
Processed PG successfully
Processed TMUS successfully
Processed CVX successfully
Processed SAP successfully
Processed ABBV successfully
Processed RCIT successfully
Processed CRM successfully
Processed KO successfully
Processed CSCO successfully
Processed MRK successfully
Processed ASML successfully
Processed WFC s

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BCH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BCH&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BJ?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BJ&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/UHAL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=UHAL&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/q

KeyboardInterrupt: 

In [14]:
# Initialize Pinecone client with the API key from user data
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"))

# Connect to the specified Pinecone index
pinecone_index = pc.Index(index_name)


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GIL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GIL&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WBA?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=WBA&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GRFS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GRFS&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance

In [15]:
query = "What are some companies that manufacture consumer hardware?"

In [16]:
raw_query_embedding = get_huggingface_embeddings(query)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MIDD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MIDD&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/TREX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=TREX&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CVLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CVLT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

In [17]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LOAR?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LOAR&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/OUKPY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=OUKPY&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AXS?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AXS&crumb=Edge%3A+Too+Many+Requests


In [18]:
top_matches

{'matches': [{'id': 'e3235cbd-609a-465a-98a4-86d66c6c0a64',
              'metadata': {'Business Summary': '3M Company provides '
                                               'diversified technology '
                                               'services in the United States '
                                               'and internationally. The '
                                               "company's Safety and "
                                               'Industrial segment offers '
                                               'industrial abrasives and '
                                               'finishing for metalworking '
                                               'applications; autobody repair '
                                               'solutions; closure systems for '
                                               'personal hygiene products, '
                                               'masking, and packaging '
                                

In [19]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [20]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [21]:
print(augmented_query)

<CONTEXT>
3M Company provides diversified technology services in the United States and internationally. The company's Safety and Industrial segment offers industrial abrasives and finishing for metalworking applications; autobody repair solutions; closure systems for personal hygiene products, masking, and packaging materials; electrical products and materials for construction and maintenance, power distribution, and electrical original equipment manufacturers; structural adhesives and tapes; respiratory, hearing, eye, and fall protection solutions; and natural and color-coated mineral granules for shingles. Its Transportation and Electronics segment provides ceramic solutions; attachment/bonding products, films, sound, and temperature management for transportation vehicles; premium large format graphic films for advertising and fleet signage; light management films and electronics assembly solutions; packaging and interconnection solutions; semiconductor production materials; data cen

In [22]:
%%writefile .env
GOOGLE_API_KEY = your api key

Writing .env


ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/OSK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=OSK&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GGB?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GGB&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ENLC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ENLC&crumb=Edge%3A+Too+Many+Requests


In [23]:
from dotenv import load_dotenv
load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FRHC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FRHC&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/GXO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=GXO&crumb=Edge%3A+Too+Many+Requests


In [24]:
def generate_explaination():
  prompt = f"""Task:
Based on the context provided below, identify companies relevant to the topic of the prompt and provide a structured explanation of their roles. Ensure the response is directly related to the user's question and format it as a clear and concise list.

Example Output Format:

Company Name: Brief explanation of how the company relates to the topic of the question.
Company Name: Brief explanation of their contribution or relevance to the topic.
Context:

[Insert your entire context here.]

Question:

[Insert your specific question here.]
"""

  model = genai.GenerativeModel(model_name="gemini-1.5-flash")
  response = model.generate_content([prompt,augmented_query])

  return response.text

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/QFIN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=QFIN&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CUZ?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CUZ&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MYTHY?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MYTHY&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/fin

In [25]:
print(generate_explaination())

ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/COLM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=COLM&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SEM?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SEM&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PRCT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PRCT&crumb=Edge%3A+Too+Many+Requests
ERROR:yfinance:429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finan

* **3M Company:** Manufactures consumer hardware such as bandages, braces, supports, respirators, home cleaning products, retail abrasives, and stationery products.

* **Best Buy Co., Inc.:** While primarily a retailer, Best Buy sells a wide range of consumer hardware including computers, mobile phones, tablets, smartwatches, home theater systems, appliances, and other consumer electronics.  They don't *manufacture* these, but they are a significant distributor of consumer hardware.

* **Corning Incorporated:** Manufactures consumer hardware components, specifically glass substrates for flat panel displays used in televisions, computers, tablets, and other devices.

* **Honeywell International Inc.:** Produces some consumer-facing hardware, though a smaller portion of their business, within their Safety and Productivity Solutions segment (e.g., personal protective equipment).

